# LUME-EBLT devel

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from eblt.input import EBLTInput, assign_names_to_elements
from eblt.output import EBLTOutput
from eblt.plot import plot_lattice_lines

import numpy as np

from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.units import mec2
import rich
import matplotlib.pyplot as plt

# Load input file

In [ ]:
input = EBLTInput.from_file("example1/eblt.in")
assign_names_to_elements(input.lattice_lines)

# View input

In [ ]:
rich.print(input)

# Plot lattice

In [ ]:
plot_lattice_lines(input.lattice_lines)

# Write input for testing

TODO: parsing rfdata, pts.in

In [ ]:
input.to_file("test/eblt.in")

In [ ]:
!mkdir test
!cd test;cp ../example1/rfdata* .;cp ../example1/pts.in .;

# Run manually

In [ ]:
!cd test;$EBLT_EXE >log.txt;head log.txt;tail log.txt

# Load output

In [ ]:
output = EBLTOutput.from_directory("test")
output.stats.model_fields.keys()

In [ ]:
output.stats.model_fields

In [ ]:
fig, ax = plt.subplots()
ax.plot(output.stats.distance, output.stats.kinetic_energy / 1e6, color="red")
ax.set_xlabel(r"$s$ (m)")
ax.set_ylabel(r"$E_\text{kinetic}$ (MeV)")
ax.set_ylim(0, None)

In [ ]:
fig, ax = plt.subplots()
ax.plot(output.stats.distance, output.stats.rms_z * 1e3, color="blue")
ax.set_xlabel(r"$s$ (m)")
ax.set_ylabel(r"$\sigma_z$ (mm)")
ax.set_ylim(0, None)

# Particles 

In [ ]:
p = output.particle_distributions[201]
plt.scatter(p.z, p.delta_gamma)

In [ ]:
def particlegroup_data_from_eblt(p):
    z = p.z
    gamma = p.gamma
    weight = p.weight
    n = len(z)
    pz = np.sqrt(gamma**2 - 1) * mec2
    return dict(
        t=np.zeros(n),
        x=np.zeros(n),
        px=np.zeros(n),
        y=np.zeros(n),
        py=np.zeros(n),
        z=p.z,
        pz=pz,
        weight=weight,
        status=np.ones(n),
        species="electron",
    )


P = ParticleGroup(data=particlegroup_data_from_eblt(p))

P.plot("z", "energy", bins=100)